In [2]:
import os
import base64
import re
import csv
from dotenv import load_dotenv, find_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import *
from azure.search.documents.models import VectorizedQuery
import openai
from openai import AzureOpenAI

In [3]:
# Get Environment settings from .env file
load_dotenv(find_dotenv())

# Azure AI Search Index Settings
service_endpoint = f"{os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT')}"
index_creds = AzureKeyCredential(os.getenv("AZURE_SEARCH_INDEX_KEY"))
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME_TEXT")

## Create a client for querying the index
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=index_creds)
## Create an index
index_client = SearchIndexClient(service_endpoint, index_creds)

# Azure Openai Settings
openai.api_type = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.azure_endpoint = os.getenv("OPENAI_API_ENDPOINT")
openai.api_version = os.getenv("OPENAI_API_VERSION")

azure_openai_client = AzureOpenAI(
    api_key = os.getenv("OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("OPENAI_API_ENDPOINT")
)

In [4]:
def get_embedding(text, model="textembedding"): # model=[Deployment Name], DONOT change this
   text = text.replace("\n", " ")
   return azure_openai_client.embeddings.create(input = [text], model=model).data[0].embedding

In [8]:
sections = []
with open('ch1to3.csv', 'rt', newline='', encoding='utf-8', errors='ignore') as csvfile:
    csvreader = csv.reader(csvfile)
    for item in csvreader:
        section = {
            "id": f"{item[0]}-{item[1]}-{item[2]}",
            "Chapter": item[0],
            "Section": item[1],
            "Paragraph": item[2],
            "Content": item[3],
            "Embedding": get_embedding(item[3]),
        }
        sections.append(section)
print(f"Finished Indexing: {len(sections)} items in total")


Finished Indexing: 101 items in total


In [11]:
index = SearchIndex(
    name=index_name,
    fields=[
        SimpleField(name="id", type="Edm.String", key=True),
        SearchableField(name="Chapter", type="Edm.String", analyzer_name="standard.lucene", 
                        filterable=True, sortable=True, facetable=True, searchable=True),
        SearchableField(name="Section", type="Edm.String", analyzer_name="standard.lucene",
                        filterable=True, sortable=True, facetable=True, searchable=True),
        SearchableField(name="Paragraph", type="Edm.String", analyzer_name="standard.lucene",
                        filterable=True, sortable=True, facetable=True, searchable=True),        
        SearchableField(name="Content", type="Edm.String", analyzer_name="standard.lucene",
                        filterable=True, sortable=True, facetable=True, searchable=True),
        SearchField(name="Embedding", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),  
            hidden=False, searchable=True, filterable=False, sortable=False, facetable=False,
            vector_search_dimensions=1536, vector_search_profile_name="my-vector-config"),
    ],
    vector_search=VectorSearch(
        profiles=[VectorSearchProfile(
            name="my-vector-config",
            algorithm_configuration_name="my-hnsw")
        ],
        algorithms=[
            HnswAlgorithmConfiguration(name="my-hnsw")
        ]
    )
)

index_client.create_or_update_index(index)

In [12]:
results = search_client.upload_documents(documents=sections)
print("Uploading")
succeeded = sum([1 for r in results if r.succeeded])
print(f"Indexed {len(results)} sections, {succeeded} succeeded")
batch = []

Uploading
Indexed 101 sections, 101 succeeded


In [13]:
query = "Who is Francis May? What did he/she do related to HKU?" #your query keywords
query_vector = get_embedding(query)

r = search_client.search(
    search_text=None,
    top=7,
    vector_queries=[VectorizedQuery(
        vector=query_vector,
        fields="Embedding"
    )]
)

search_results = []
for result in r:
    print("#########################################")
    print("Source: " + result["id"])
    print("Caption" + result["Content"])
    search_results.append("Source: " + result["id"] + "; Content: " + result["Content"])

#########################################
Source: 3-6-4
CaptionIts Rector, Francis May, was acting as Governor when the bill of incorpora- tion was read in Legislative Council. Ho Kai moved the readings during one week, and announced that he would shortly be in a position to get the buildings started. His motion was seconded by Wei Yuk, who later himself became associated with the University project. The opportunity was taken to remove the words 'for the Chinese' from the title the College had borne from its inauguration, since several of the students were not Chinese. May advanced his opinion that there was perhaps no more useful institution in the Colony than the Hong Kong College of Medicine. The incorporation became effective on the 23rd May 1907.
#########################################
Source: 1-1-17
CaptionSome graduates are overseas. The number is comparatively small, and they have achieved in proportion the same distinction and engaged themselves in enriching the community li

In [14]:
systemMessage = """AI Assistant that helps user to answer questions from sources provided. Be specific in your answers, with supporting evidence from only the sources below.
                    Answer ONLY with the facts listed in the list of sources below. 
                    If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. 
                    Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. 
                    Use square brackets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].
                """

messages = [
    {'role' : 'system', 'content' : systemMessage},
    {'role' : 'user', 'content' : query + "   Source:" + " ".join(search_results)}
]

chat_completion = openai.chat.completions.create(
    model="summer", # Do not edit this. model="deployment_name"
    messages=messages, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)

chat_content = chat_completion.choices[0].message.content
print(chat_content)

Francis May was the Rector of the Hong Kong College of Medicine. He played a significant role in the incorporation of the college, acting as Governor when the bill of incorporation was read in the Legislative Council. He also expressed his opinion that the Hong Kong College of Medicine was perhaps the most useful institution in the Colony. The incorporation became effective on the 23rd May 1907 [3-6-4].

Additionally, Francis May was involved in the discussions and planning of the University project. He was a part of the Committee of Management that was appointed to stay in session until a University was in being. This committee comprised twelve members, including May [3-8-1].


- Why is HKU established?
- The University of Hong Kong (HKU) was established in direct response to the explicit needs of China for modern technology and a clear understanding of the Western temperament. Its original aim was to supply China with these resources, although its role has since evolved to supplying versatile citizens, civil servants, and professionals with a high level of cultural and social awareness to Hong Kong [2-1-1]. Lord William's and Sir Robert Hart's opinions also played a part in the founding of HKU. Despite Lord William's skepticism about a university in Hong Kong satisfying the national aspirations of the Chinese due to the language of instruction, Sir Robert Hart viewed the Hong Kong scheme as more 'practically promising' [3-11-4].

### How is Taiping Rebellion related to HKU's establishment?

The Taiping Rebellion is related to the establishment of the University of Hong Kong (HKU) in several ways. First, the suppression of the Taiping Rebellion in 1864 marked the beginning of China's intellectual renascence, marking the start of a dawning awareness that China must learn how to measure up to the West if she was to defend herself against aggression and the partition of her vast potential among foreign powers [2-1-3]. This led to a scramble for foreign powers to establish educational footholds in China [2-1-2]. 

Second, the Taiping Rebellion and its aftermath resulted in an influx of reformists into Hong Kong, many of whom had close connections with the Christian missions [2-6-3]. Notable among them was K'ang Yu-wei, who, after the collapse of his reform movement in China, fled to Hong Kong and began a long exile. It was during this period that the Dowager Empress initiated the foundation of the first modern Chinese university and secondary schools in some provincial capitals, which were to be developed into a system of National Universities on the Imperial University model [2-6-3].

Furthermore, the Taiping Rebellion led to increased interaction between China and foreign powers, which resulted in increased foreign influence in China, including Hong Kong. This foreign influence played a key role in the establishment of HKU. For instance, Sir Frederick Lugard and others founded HKU in 1911 to help in a Chinese renascence as part of a more general competition among foreign powers interested in securing strong educational footholds in China [2-1-2].

Finally, the fund-raising activities for the establishment of HKU were also influenced by the events following the Taiping Rebellion. Among the fund-raising activities was a performance dedicated to Prince Hsiun, who was on a tour of inspection of China's naval installations as part of the imperial modernization programme initiated in the wake of the Taiping Rebellion [3-12-5]. Prince Hsiun expressed his support for the University project, emphasizing the role of education in promoting trade, friendly feelings, and providing the poorer classes of China's population with a useful modern education [3-12-5]


The Taiping Rebellion is related to the establishment of the University of Hong Kong (HKU) in that the suppression of the rebellion marked the beginning of China's intellectual renascence and increased Western influence in China [2-1-3]. Following the suppression of the rebellion, Britain and France were granted travel concessions, extraterritorial rights, and permission to make Christian converts among the Chinese [2-1-3]. This led to a scramble for footholds in the Chinese system by foreign powers, including the establishment of educational institutions [2-1-2][2-1-3]. 

Furthermore, in the wake of the Rebellion, there was a realization that China needed to modernize and come to terms with the West [2-1-3]. One of the ways this was done was through the founding of the first modern Chinese university - the Imperial University [2-6-3]. The establishment of HKU was part of this wider movement to modernize China's education system and to equip the Chinese populace with modern education [2-1-2][3-12-5]. 

In addition, several prominent reformers who were involved in the establishment of HKU had links to the aftermath of the Taiping Rebellion. For instance, K'ang Yu-wei, a Cantonese reformer, fled to Hong Kong after the collapse of his Reform Club [2-6-3]. Sun Yat-sen, who was among the first group enrolled in the College of Medicine in Hong Kong, was in contact with young radical reformers who belonged to the Sanhohui, a large anti-Manchu triad in Canton [2-8-3]. This connection shows that the intellectual and political climate following the rebellion played a role in the establishment of HKU

### Who is Francis May?
Francis May was the Rector of the Hong Kong College of Medicine. He played a significant role in the incorporation of the college, acting as Governor when the bill of incorporation was read in the Legislative Council. He also expressed his opinion that the Hong Kong College of Medicine was perhaps the most useful institution in the Colony. The incorporation became effective on the 23rd May 1907 [3-6-4].

Additionally, Francis May was involved in the discussions and planning of the University project. He was a part of the Committee of Management that was appointed to stay in session until a University was in being. This committee comprised twelve members, including May [3-8-1].